### Imports

In [1]:
import re
import boto3
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

### Data download

In [13]:
s3 = boto3.client("s3")
s3.download_file(
    Bucket="cheapper", Key="webscraping_results_17-01-2023.csv", Filename="data.csv"
)

### Read data

In [2]:
df = pd.read_csv('data.csv').drop('Unnamed: 0',axis = 1)

In [3]:
df = df[df['name'].str.contains('yerba',case = False)].reset_index(drop = True) # me quedo solo con yerba

In [15]:
df.loc[[69,72,104,176,246]]

,date,ecomm_name,name,src,price_1,price_2
69,17-01-2023,elclon,YERBA CANARIAS 1 KG,https://f.fcdn.app/imgs/95fec6/www.elclon.com....,185,NaN
72,17-01-2023,eldorado,YERBA CANARIAS 1KG,https://eldoradouy.vtexassets.com/arquivos/ids...,"188,00",NaN
104,17-01-2023,devoto,Yerba CANARIAS 1 kg,https://geant.vteximg.com.br/arquivos/ids/2921...,188,NaN
176,17-01-2023,disco,Yerba CANARIAS 1 kg,https://geant.vteximg.com.br/arquivos/ids/2921...,188,NaN
246,17-01-2023,tiendainglesa,Yerba CANARIAS 1 Kg,https://images-ti-vm1.tiendainglesa.com.uy/lar...,190,NaN


### Unsupervised clasification

In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\emanuel.rieiro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
def containsNumber(value):
    for character in value:
        if character.isdigit():
            return True
    return False

In [16]:
def remove_stopwords(texts):
    stop_words = set(stopwords.words('spanish'))
    stop_words.add('gr')
    stop_words.add('g')
    stop_words.add('grs.')
    stop_words.add('grs')
    stop_words.add('kg')
    stop_words.add('kg.')
    stop_words.add('&')
    stop_words.add('+')
    stop_words.add('.')
    stop_words.add('%')
    stop_words.add('clasica')
    cleaned_texts = []
    for text in texts:
        cleaned_text = [word for word in text.split() if word.lower() not in stop_words]
        cleaned_texts.append(' '.join(cleaned_text))
    return cleaned_texts

In [17]:
def remove_stopwords_together(texts):
    
    new_names = []
    for names in texts:
        
        new_name = []
        for name in names.split():
            
            if containsNumber(name):
                new_name.append(re.findall(r'\d+', name)[0])
            else:
                new_name.append(name)
        
        new_names.append(' '.join(new_name))
    return new_names

In [18]:
def remove_yerba_mate(texts):
    if 'yerba' in texts.split() and 'mate' in texts.split():
            return texts.replace('mate ','')
    else:
        return texts

In [19]:
df['name'] = df['name'].astype(str) # get everything to str
df['name'] = df['name'].str.lower() # lowercase everything
df['name'] = df['name'].str.replace('c/','') # get rid of the c/
df['name'] = df['name'].str.replace('(','')
df['name'] = df['name'].str.replace(')','')
df['name'] = df['name'].str.replace('á','a')
df['name'] = df['name'].str.replace('é','e')
df['name'] = df['name'].str.replace('í','i')
df['name'] = df['name'].str.replace('ó','o')
df['name'] = df['name'].str.replace('ú','u')

df['name'] = remove_stopwords(df['name'].values) # stopwords
df['name'] = remove_stopwords_together(df['name']) # clean "500g" like descriptions
df['name'] = [remove_yerba_mate(descriptions) for descriptions in df['name']] # remove mate from yerba mate

C:\Users\emanuel.rieiro\AppData\Local\Temp\ipykernel_11588\3627394488.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['name'] = df['name'].str.replace('(','')
C:\Users\emanuel.rieiro\AppData\Local\Temp\ipykernel_11588\3627394488.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['name'] = df['name'].str.replace(')','')


In [20]:
df.loc[[69,72,104,176,246]]

,date,ecomm_name,name,src,price_1,price_2
69,17-01-2023,elclon,yerba canarias 1,https://f.fcdn.app/imgs/95fec6/www.elclon.com....,185,NaN
72,17-01-2023,eldorado,yerba canarias 1,https://eldoradouy.vtexassets.com/arquivos/ids...,"188,00",NaN
104,17-01-2023,devoto,yerba canarias 1,https://geant.vteximg.com.br/arquivos/ids/2921...,188,NaN
176,17-01-2023,disco,yerba canarias 1,https://geant.vteximg.com.br/arquivos/ids/2921...,188,NaN
246,17-01-2023,tiendainglesa,yerba canarias 1,https://images-ti-vm1.tiendainglesa.com.uy/lar...,190,NaN


### Text classification

In [21]:
def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

In [22]:
def create_bow(texts):
    vectorizer = CountVectorizer()
    bow = vectorizer.fit_transform(texts)
    return bow, vectorizer

In [23]:
# method 1: Lo puedo seguir explorando mañana en clase
from sklearn.feature_extraction.text import CountVectorizer

cv =  CountVectorizer(tokenizer=lambda txt: txt.split()) # tengo que usar una función custom
bow = cv.fit_transform(df['name'])

count_array = bow.toarray()
df_features = pd.DataFrame(data=count_array, columns = cv.get_feature_names_out())

C:\Users\emanuel.rieiro\Anaconda3\envs\cheapper\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [24]:
print('Item:', df['name'].loc[69])
print('')
print('Unos en el vector:')
df_features.loc[:,df_features.loc[69] == 1].loc[69]

Item: yerba canarias 1

Unos en el vector:


1           1
canarias    1
yerba       1
Name: 69, dtype: int64

In [341]:
distances = np.zeros((len(df_features), len(df_features)), dtype='float32')

for i in df_features.index:
    for u in df_features.index:
        distances[i, u] = np.linalg.norm(df_features.loc[i] - df_features.loc[u])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [343]:
# Manteniendo valores unicos
df_distances = pd.DataFrame(distances, columns = (df['ecomm_name'] + '|' + df['name']).values, index = df['name'].values)
df_distances = df_distances.reset_index().drop_duplicates()
df_distances = df_distances.set_index('index')

In [344]:
df_distances.loc['yerba cebador 70',df_distances.loc['yerba cebador 70'] <= 1].sort_values()

elclon|yerba cebador 70              0.0
devoto|yerba cebador 70              0.0
disco|yerba cebador 70               0.0
elclon|yerba cebador compuesta 70    1.0
elclon|yerba cebador intenso 70      1.0
devoto|yerba cebador compuesta 70    1.0
disco|yerba cebador compuesta 70     1.0
Name: yerba cebador 70, dtype: float32

In [233]:
def get_item(item):
    print(df_distances.loc[item,df_distances.loc[item] <= 1].sort_values())

In [257]:
text = 'Yerba c/ canarias'

text = text.lower() # lowercase everything
text = text.replace('c/','') # get rid of the c/
text = text.replace('(','')
text = text.replace(')','')
text = text.replace('á','a')
text = text.replace('é','e')
text = text.replace('í','i')
text = text.replace('ó','o')
text = text.replace('ú','u')

text = remove_stopwords([text]) # stopwords
text = remove_stopwords_together(text) # clean "500g" like descriptions
text = [remove_yerba_mate(descriptions) for descriptions in text] # remove mate from yerba mate

asd = cv.transform(text).toarray()

In [275]:
asd_distances = []
for feature in df_features.index:
    asd_distances.append(np.linalg.norm(asd - df_features.loc[feature].values))

In [286]:
df_features.drop_duplicates()

,0,1,10,100,105,108,110,114,115,116,...,walkers,wasa,watercolor,x,xl,yerba,yogur,yogurt,zabet,zapallo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [346]:
df.to_excel('Index depurado.xlsx', index = False)